# raw_to_metadata monthly data

take the most recent month's mbox file and convert it into a csv for analysis.

In [ ]:
import mailbox
import os 
import gzip
import pandas as pd
import boto3
from pathlib import Path
from dotenv import load_dotenv
load_dotenv("../../.env")

In [ ]:
# TODO - add bits for running in automation

In [ ]:
path = Path("../../data/raw/fedora-devel-list/")
mboxs = [x.name for x in list(path.glob('*.mbox'))]

In [ ]:
def mbox_to_meta(mbox):
    
    csv = []
    for msg in mbox:
        msg_id = msg["Message-ID"]
        date = msg["Date"]
        from_ = msg["From"]
        to = msg["To"]
        subject = msg["Subject"]
        reply_to = msg["In-Reply-To"]

        csv.append((msg_id, date, from_, to, subject, reply_to))
    df = pd.DataFrame(csv,columns=['Message-ID', 'Date', 'From',
                                       'To', 'Subject', 'In-Reply-To'])
        
    return df

In [ ]:
base = "/".join(path.relative_to(".").parts)
for mbox in mboxs:
    monthly_mbox = "/".join([base,mbox])
    monthly_mbox = mailbox.mbox(monthly_mbox)
    df = mbox_to_meta(monthly_mbox)
    df.to_csv("../../data/interim/metadata/"+mbox+".csv")
    print(f"data/interim/metadata/{mbox} saved")

In [ ]:
# Push to Ceph

In [ ]:
BASE_PATH = os.getenv("LOCAL_DATA_PATH", "../../data")

S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL', 'https://s3.upshift.redhat.com')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
S3_BUCKET = os.getenv('S3_BUCKET', 'DH-PLAYPEN')
S3_PROJECT_KEY = os.getenv("S3_PROJECT_KEY","'mcliffor/fedora_devel_mail/") 